In [89]:
import json
import os
import pandas as pd
from pycocotools.coco import COCO
from sklearn.metrics import cohen_kappa_score
import numpy as np

In [90]:
# Function to load a JSON file into a Python dictionary
def load_json(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)

In [91]:
def calculate_iou(box1, box2):
    # Extract x, y, w, h from the input boxes
    
    x1 = box1[0]
    y1 = box1[1] 
    w1 = box1[2] 
    h1 = box1[3]
    
    x2 = box2[0]
    y2 = box2[1] 
    w2 = box2[2] 
    h2 = box2[3]
    
    # Determine the coordinates of the intersection rectangle
    x_intersect = max(x1, x2)
    y_intersect = max(y1, y2)
    w_intersect = min(x1 + w1, x2 + w2) - x_intersect
    h_intersect = min(y1 + h1, y2 + h2) - y_intersect
    
    # Ensure the intersection width and height are non-negative
    w_intersect = max(0, w_intersect)
    h_intersect = max(0, h_intersect)
    
    # Compute the area of intersection rectangle
    interArea = w_intersect * h_intersect
    
    # Compute the area of both the prediction and true bounding boxes
    box1Area = w1 * h1
    box2Area = w2 * h2
    
    # Compute the area of union
    unionArea = box1Area + box2Area - interArea
    
    # Compute the Intersection over Union by dividing the intersection area by the union area
    iou = interArea / unionArea if unionArea > 0 else 0.0
    
    return iou

In [92]:
def get_coco_maps(f1, f2):

    coco1 = COCO(f1)
    coco2 = COCO(f2)

    coco1_map = {}
    images1 = coco1.loadImgs(coco1.getImgIds())
    for img1 in images1:
        img1_ann_list = []
        bn = os.path.basename(img1["file_name"])
        img1_fn = bn.split("-")[-2] + "-" + bn.split("-")[-1]
        anns1 = coco1.loadAnns(coco1.getAnnIds(img1["id"]))
        for ann1 in anns1:
            img1_ann_list.append([ann1["bbox"], ann1["category_id"]])
        coco1_map[img1_fn] = img1_ann_list
        
    coco2_map = {}
    images2 = coco2.loadImgs(coco2.getImgIds())
    for img2 in images2:
        img2_ann_list = []
        bn = os.path.basename(img2["file_name"])
        img2_fn = bn.split("-")[-2] + "-" + bn.split("-")[-1]
        anns2 = coco2.loadAnns(coco2.getAnnIds(img2["id"]))
        for ann2 in anns2:
            img2_ann_list.append([ann2["bbox"], ann2["category_id"]])
        coco2_map[img2_fn] = img2_ann_list
        
    return coco1_map, coco2_map


In [93]:
def get_master_list(cm1, cm2):

    master_list = []

    for key in cm1.keys():
        data = []

        for j, ann1 in enumerate(cm1[key]):
            for k, ann2 in enumerate(cm2[key]):
                if ann1[1] == ann2[1]:  
                    iou_score = calculate_iou(ann1[0], ann2[0])  
                    if iou_score > thold:
                        data.append({
                            'iou_score': iou_score,
                            'index1': j,
                            'index2': k
                        })

        df = pd.DataFrame(data, columns=['iou_score', 'index1', 'index2'])
        df = df.sort_values(by='iou_score', ascending=False)
        min_len = min(len(cm1[key]), len(cm2[key]))
        max_len = max(len(cm1[key]), len(cm2[key]))
        df = df.head(min_len).reset_index(drop=True)
        master_list.append([key, min_len, max_len, df])

    return master_list


In [94]:
def write_metrics_to_file(master_list, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        
    acc_list = []
    avg_iou_list = []

    output_file = os.path.join(output_directory, "metrics.txt")
    with open(output_file, "w") as file:
        file.write("PER IMAGE METRICS\n")
        for img in master_list:
            file.write(f"{img[0]}\n")
            acc = len(img[3])/img[2]
            acc_list.append(acc)
            file.write(f"   accuracy: {acc}\n")
            avg_iou = np.mean(img[3]["iou_score"])
            avg_iou_list.append(avg_iou)
            file.write(f"   avg iou: {avg_iou}\n")

        file.write("METRICS OVERALL\n")
        file.write(f"   avg accuracy for all: {np.mean(acc_list)}\n")
        file.write(f"   avg iou for all: {np.mean(avg_iou_list)}\n")

In [95]:
in_f1 = "/mnt/lts/nis_lab_research/data/coco_files/parts/coco/shah_b7_65454/result.json"
in_f2 = "/mnt/lts/nis_lab_research/data/coco_files/parts/coco/shah_b7_65454/result.json"

out_dir = "./output"

thold = .5


In [96]:
coco1_map, coco2_map = get_coco_maps(in_f1, in_f2)
master_list = get_master_list(coco1_map, coco2_map)
write_metrics_to_file(master_list, out_dir)

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
